In [5]:
import numpy as np
# Linear Regression class with (OLS, Normal Equation and Gradient Descent)
class LinearRegression():

    # Initialization
    def __init__(self, method = 'OLS', learning_rate = 0.1, n_iterations = 1000, alpha=1.0):
        self.method = method
        self.learning_rate = learning_rate
        self.n_iterations = n_iterations
        self.alpha = alpha
        self.theta = None

    # Fit 
    def fit(self, X, y):
        # Get number of rows and columns of training data
        m, n=X.shape

        # Add bias => Xo = 1 to training data
        X_b = np.c_[np.ones((m,1)), X]

        # Get theta by each method (calculate paramerts to apply it within function in predict)
        if self.method == 'OLS':
            self.theta = np.linalg.solve(X_b.T @ X_b, X_b.T @ y)

        elif self.method == 'Normal':
            self.theta = np.linalg.inv(X_b.T @ X_b) @ X_b.T @ y

        elif self.method == 'Gradient Descent':
            # It is n+1 because we have theta 0 
            self.theta = np.random.randn(n + 1)   
            for iteration in range(self.n_iterations):
                h_theta = X_b @ self.theta
                gradients = 2/m * X_b.T @ (h_theta - y)
                self.theta -= self.learning_rate * gradients

        elif self.method == 'Ridge':
            # I matrix 
            I = np.eye(n+1)
            # No regularization of bias
            I[0,0] = 0  
            self.theta = np.linalg.inv(X_b.T @ X_b + self.alpha * I) @ X_b.T @ y

        elif self.method == 'Ridge-Gradient Descent':
            # It is n+1 because we have theta 0 
            self.theta = np.random.randn(n + 1)   
            for iteration in range(self.n_iterations):
                h_theta = X_b @ self.theta
                # Add regularization term and make sure not included theta 0 
                gradients = 2/m * ( X_b.T @ (h_theta - y) + self.alpha * np.r_[0, self.theta[1:]] )
                self.theta -= self.learning_rate * gradients  

        elif self.method == 'Lasso-Gradient Descent':
            # It is n+1 because we have theta 0 
            self.theta = np.random.randn(n+1)
            for _ in range(self.n_iterations):
                h_theta = X_b @ self.theta
                gradients = 2/m * X_b.T @ (h_theta - y)
                # Add regularization term nd make sure not included theta 0 and never update bias dervative
                gradients[1:] += self.alpha * np.sign(self.theta[1:])
                self.theta -= self.learning_rate * gradients        

        else :
            raise ValueError('Unknown method.')
        
        # Bias value and other theta values 
        self.intercept_ = self.theta[0]
        self.coef_ = self.theta[1:]
        
    # Predict
    def predict(self, X):
        # Get number of rows of test data
        m = X.shape[0]

        # Add bias => Xo = 1 to test data
        X_b = np.c_[np.ones((m,1)), X]

        # Apply linear regression equation
        y_predict = X_b @ self.theta
        return y_predict
    
    # Score 
    def score(self, X_new, y):
        # Return R^2 score of the model
        y_pred = self.predict(X_new)
        ss_total = np.sum((y - np.mean(y)) **2)
        ss_residual = np.sum((y - y_pred) **2)
        return 1 - ss_residual / ss_total


In [ ]:
# Sample data
# Create number of rows and random x and y matrices
m = 100
# Use rand then multiple by 2 to make sure the samples values are between 0 and 2 this would make sure we simulate feature scaling
X = 2 * np.random.rand(m, 1)
# Y value will split to intercept + value + noise doing regression
y = 4 + 3 * X[:, 0] + np.random.randn(m)

# Test data with 2 rows
X_new = np.array([[0], [5], [20], [66]])

# Apply linear regression with all possible methods
methods = ['OLS', 'Normal', 'Gradient Descent', 'Ridge', 'Ridge-Gradient Descent', 'Lasso-Gradient Descent', 'Error 404']
for method in methods:
    # Get linear regression object
    lr = LinearRegression(method)
    lr.fit(X, y)
    y_pred = np.round(lr.predict(X_new), 2)
    r2_train = lr.score(X, y) 
    print(f'Method: {method}')
    print(f'Predictions for {X_new.tolist()}: {y_pred}')
    print(f'R^2 score on training data: {r2_train:0.2f} %')
    print(f'Intercept: {lr.intercept_:.2f}')
    print(f'Coefficient: {np.round(lr.coef_, 2)}\n')
    print('-' * 40)

Method: OLS
Predictions for [[0], [5], [20], [66]]: [  4.27  18.36  60.64 190.28]
R^2 score on training data: 0.75 %
Intercept: 4.27
Coefficient: [2.82]

----------------------------------------
Method: Normal
Predictions for [[0], [5], [20], [66]]: [  4.27  18.36  60.64 190.28]
R^2 score on training data: 0.75 %
Intercept: 4.27
Coefficient: [2.82]

----------------------------------------
Method: Gradient Descent
Predictions for [[0], [5], [20], [66]]: [  4.27  18.36  60.64 190.28]
R^2 score on training data: 0.75 %
Intercept: 4.27
Coefficient: [2.82]

----------------------------------------
Method: Ridge
Predictions for [[0], [5], [20], [66]]: [  4.37  18.01  58.93 184.41]
R^2 score on training data: 0.75 %
Intercept: 4.37
Coefficient: [2.73]

----------------------------------------
Method: Ridge-Gradient Descent
Predictions for [[0], [5], [20], [66]]: [  4.37  18.01  58.93 184.41]
R^2 score on training data: 0.75 %
Intercept: 4.37
Coefficient: [2.73]

-----------------------------

ValueError: Unknown method.